### `DBSCAN`

The DBSCAN algorithm views clusters as areas of high density separated by areas of low density. Due to this rather generic view, clusters found by DBSCAN can be any shape, as opposed to k-means which assumes that clusters are convex shaped. The central component to the DBSCAN is the concept of core samples, which are samples that are in areas of high density. **A cluster is therefore a set of core samples, each close to each other and a set of non-core samples that are close to a core sample**.

There are two parameters to the algorithm, `min_samples` and `eps`, which define formally what we mean when we say dense. Higher min_samples or lower eps indicate higher density necessary to form a cluster. More formally, **we define a core sample as being a sample in the dataset such that there exist min_samples other samples within a distance of eps, which are defined as neighbors of the core sample**.

While the parameter `min_samples` primarily controls how tolerant the algorithm is towards noise (on noisy and large data sets it may be desirable to increase this parameter), the parameter `eps` is crucial to choose appropriately for the data set and distance function and usually cannot be left at the default value. It controls the local neighborhood of the points. When chosen too small, most data will not be clustered at all (and labeled as -1 for “noise”). When chosen too large, it causes close clusters to be merged into one cluster, and eventually the entire data set to be returned as a single cluster. Some heuristics for choosing this parameter have been discussed in the literature, for example based on a knee in the nearest neighbor distances plot

#### **Example: Synthesis Dataset**

#### **Building a DBSCAN using** `scikit-learn`

We now have all of the pieces to make DBSCAN using scikit learn library.

We initialize sklearn.cluster.DBSCAN and fit the density-based clustering from features or distance matrix

One can access the labels assigned by DBSCAN using the labels_ attribute.Noisy samples are given the label math: -1.

Core samples (large dots) and non-core samples (small dots) are color-coded according to assigned cluster. Samples tagged as noise are represented in black.

`Optimizing minimum samples and epsilon`